In [4]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

from joblib import Parallel, delayed

In [16]:
df = pd.read_fwf('Abell_eq.dat.txt')
df.columns = ['name', 'ra', 'dec']
objects = df.to_dict('records')

for obj_idx in range(len(object_names)):
    hh, mm, ss = [x.strip().zfill(2) for x in objects[obj_idx]['ra'].split(':')]
    objects[obj_idx]['ra'] = f'{hh}h{mm}m{ss}s'
    
    dd, mm = [x.strip().zfill(2) for x in objects[obj_idx]['dec'].split(':')]
    objects[obj_idx]['dec'] = f'{dd}d{mm}m00s'

objects

# with open('Abell_eq.dat.txt') as f:
#     object_names = [{
#         'name': line[:-1].split('\t')[0],
#         'glon': np.float64(line[:-1].split('\t')[1]),
#         'glat': np.float64(line[:-1].split('\t')[2])
#     } for line in f]
# object_names[:3]

[{'name': 'ABELL3299', 'ra': '04h10m42s', 'dec': '-88d44m00s'},
 {'name': 'ABELL2757', 'ra': '00h15m48s', 'dec': '-88d10m00s'},
 {'name': 'ABELL3710', 'ra': '21h05m54s', 'dec': '-87d47m00s'},
 {'name': 'ABELL3763', 'ra': '21h38m06s', 'dec': '-87d12m00s'},
 {'name': 'ABELL3595', 'ra': '14h32m24s', 'dec': '-86d55m00s'},
 {'name': 'ABELL3650', 'ra': '20h07m06s', 'dec': '-86d48m00s'},
 {'name': 'ABELL3625', 'ra': '16h13m00s', 'dec': '-86d48m00s'},
 {'name': 'ABELL3801', 'ra': '21h55m30s', 'dec': '-86d45m00s'},
 {'name': 'ABELL3333', 'ra': '04h59m30s', 'dec': '-86d42m00s'},
 {'name': 'ABELL973S', 'ra': '21h55m54s', 'dec': '-86d25m00s'},
 {'name': 'ABELL860S', 'ra': '20h29m48s', 'dec': '-86d11m00s'},
 {'name': 'ABELL1091S', 'ra': '23h13m24s', 'dec': '-85d57m00s'},
 {'name': 'ABELL791S', 'ra': '17h10m30s', 'dec': '-85d54m00s'},
 {'name': 'ABELL3037', 'ra': '02h28m48s', 'dec': '-85d50m00s'},
 {'name': 'ABELL480S', 'ra': '04h24m54s', 'dec': '-85d48m00s'},
 {'name': 'ABELL389S', 'ra': '03h35m54s

In [17]:
size = (192, 192)

In [18]:
ROOT = '../data/resaved_HPXcvt'

def perform_object(name, ra, dec):
    hdu_100 = fits.open(f'{ROOT}/Planck100.fits')[0]
    wcs_100 = WCS(hdu_100.header)
    hdu_143 = fits.open(f'{ROOT}/Planck143.fits')[0]
    wcs_143 = WCS(hdu_143.header)
    hdu_217 = fits.open(f'{ROOT}/Planck217.fits')[0]
    wcs_217 = WCS(hdu_217.header)
    hdu_353 = fits.open(f'{ROOT}/Planck353.fits')[0]
    wcs_353 = WCS(hdu_353.header)
    hdu_545 = fits.open(f'{ROOT}/Planck545.fits')[0]
    wcs_545 = WCS(hdu_545.header)

    datasets = [
        (hdu_100, wcs_100),
        (hdu_143, wcs_143),
        (hdu_217, wcs_217),
        (hdu_353, wcs_353),
        (hdu_545, wcs_545)
    ]

    obj = np.zeros((5, size[0], size[1]))
    for freq_idx, (hdu, wcs) in enumerate(datasets):
        position = SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2000.0') #  RA,deg  Dec,deg
        cutout = Cutout2D(hdu.data, position, size, wcs=wcs)
        data = np.array(cutout.data)
        if data.shape != size or np.isnan(data).any():
            return None
        obj[freq_idx] = data
    return obj, name

results = Parallel(n_jobs=6)(delayed(perform_object)(**object_name) for object_name in tqdm(objects))

In [19]:
isnone = 0
for i in results:
    if i is None:
        isnone += 1
print(isnone)

77


In [20]:
sz_name = np.array([x[1] for x in results if x is not None])
sz_data = np.stack([x[0] for x in results if x is not None])

In [21]:
np.savez_compressed('sz_abell.npz', sz_data=sz_data, sz_names=sz_name)

In [22]:
sz_name.shape

(923,)